# Quem fez o ENEM 2016 apenas para treino

___

In [1]:
# Importando as bibliotecas necessárias
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [2]:
# Carregando os datasets
treino = pd.read_csv('train.csv')
teste = pd.read_csv('test.csv')

# Selecionando algumas colunas para retirar
retira_colunas = ['NU_INSCRICAO',
#                  'IN_TREINEIRO',
#                  'CO_PROVA_CH',
#                  'CO_PROVA_LC',
#                  'CO_PROVA_MT'
                 ]
inscricao_treino = treino[['NU_INSCRICAO']]
inscricao_teste = teste[['NU_INSCRICAO']]
treineiro = treino[['IN_TREINEIRO']]
treino.drop(retira_colunas, axis=1, inplace=True)
teste.drop(retira_colunas, axis=1, inplace=True)

treino.drop('IN_TREINEIRO', axis=1, inplace=True)
# Mostrando o datasets depois da retirada das colunas
treino

,Unnamed: 0,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,...,Q041,Q042,Q043,Q044,Q045,Q046,Q047,Q048,Q049,Q050
0,1,2016,4314902,Porto Alegre,43,RS,24,M,0.0,1,...,5.0,A,A,A,A,A,A,A,B,D
1,2,2016,2304707,Granja,23,CE,17,F,0.0,3,...,NaN,A,A,C,A,B,A,A,C,A
2,3,2016,2304400,Fortaleza,23,CE,21,F,0.0,3,...,NaN,A,A,A,A,C,A,A,B,A
3,4,2016,3304557,Rio de Janeiro,33,RJ,25,F,0.0,0,...,5.0,C,A,A,A,A,D,A,A,A
4,5,2016,1302603,Manaus,13,AM,28,M,0.0,2,...,NaN,A,A,A,A,A,A,A,A,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13725,4582,2016,3502903,Araçoiaba da Serra,35,SP,20,M,0.0,1,...,NaN,A,A,A,A,A,A,A,A,A
13726,4583,2016,2613800,São Vicente Ferrer,26,PE,33,F,0.0,3,...,NaN,A,A,B,B,A,A,A,A,A
13727,4584,2016,4322186,Tupanci do Sul,43,RS,15,F,0.0,1,...,NaN,A,A,A,A,C,A,A,A,A
13728,4585,2016,3303203,Nilópolis,33,RJ,36,M,1.0,3,...,5.0,A,A,A,A,D,A,A,A,D


In [3]:
# Selecionando as variáveis numéricas do dataset de teste
numeric_features = teste.select_dtypes(include="number").columns.to_list()

# Selecioando as variáveis categóricas do dataset de teste
categoric_feature = [coluna for coluna in teste.columns if coluna not in numeric_features]

In [4]:
# Juntando todas as variáveis escolhidas
all_features = numeric_features + categoric_feature

# Aplicando as colunas escolhidas no dataset de treino
treino = treino[all_features]
treino

,CO_UF_RESIDENCIA,NU_IDADE,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,TP_DEPENDENCIA_ADM_ESC,IN_BAIXA_VISAO,...,SG_UF_RESIDENCIA,TP_SEXO,Q001,Q002,Q006,Q024,Q025,Q026,Q027,Q047
0,43,24,1,1,1,4,1,NaN,NaN,0,...,RS,M,D,D,C,A,A,C,H,A
1,23,17,3,1,2,0,2,1.0,2.0,0,...,CE,F,A,A,B,A,A,A,NaN,A
2,23,21,3,1,3,0,1,NaN,NaN,0,...,CE,F,D,D,C,A,A,A,NaN,A
3,33,25,0,1,1,9,1,NaN,NaN,0,...,RJ,F,H,E,E,C,B,C,F,D
4,13,28,2,1,1,4,1,NaN,NaN,0,...,AM,M,E,D,C,A,A,B,F,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13725,35,20,1,1,1,3,1,NaN,NaN,0,...,SP,M,E,E,G,B,B,B,C,A
13726,26,33,3,2,1,10,1,NaN,NaN,0,...,PE,F,A,A,C,A,B,B,I,A
13727,43,15,1,1,3,0,1,NaN,NaN,0,...,RS,F,C,G,B,A,B,A,NaN,A
13728,33,36,3,1,4,0,1,NaN,NaN,0,...,RJ,M,B,D,G,B,B,C,E,A


In [5]:
# Vendo a quantidade de NaN's nos dados de treino
treino.isna().sum()#.sum()

CO_UF_RESIDENCIA             0
NU_IDADE                     0
TP_COR_RACA                  0
TP_NACIONALIDADE             0
TP_ST_CONCLUSAO              0
TP_ANO_CONCLUIU              0
TP_ESCOLA                    0
TP_ENSINO                 9448
TP_DEPENDENCIA_ADM_ESC    9448
IN_BAIXA_VISAO               0
IN_CEGUEIRA                  0
IN_SURDEZ                    0
IN_DISLEXIA                  0
IN_DISCALCULIA               0
IN_SABATISTA                 0
IN_GESTANTE                  0
IN_IDOSO                     0
TP_PRESENCA_CN               0
TP_PRESENCA_CH               0
TP_PRESENCA_LC               0
TP_PRESENCA_MT               0
NU_NOTA_CN                3389
NU_NOTA_CH                3389
NU_NOTA_LC                3597
TP_LINGUA                    0
TP_STATUS_REDACAO         3597
NU_NOTA_COMP1             3597
NU_NOTA_COMP2             3597
NU_NOTA_COMP3             3597
NU_NOTA_COMP4             3597
NU_NOTA_COMP5             3597
NU_NOTA_REDACAO           3597
SG_UF_RE

In [6]:
# Vendo a quantidade de NaN's nos dados de teste
teste.isna().sum()#.sum()

CO_UF_RESIDENCIA             0
SG_UF_RESIDENCIA             0
NU_IDADE                     0
TP_SEXO                      0
TP_COR_RACA                  0
TP_NACIONALIDADE             0
TP_ST_CONCLUSAO              0
TP_ANO_CONCLUIU              0
TP_ESCOLA                    0
TP_ENSINO                 3144
TP_DEPENDENCIA_ADM_ESC    3144
IN_BAIXA_VISAO               0
IN_CEGUEIRA                  0
IN_SURDEZ                    0
IN_DISLEXIA                  0
IN_DISCALCULIA               0
IN_SABATISTA                 0
IN_GESTANTE                  0
IN_IDOSO                     0
TP_PRESENCA_CN               0
TP_PRESENCA_CH               0
TP_PRESENCA_LC               0
TP_PRESENCA_MT               0
NU_NOTA_CN                1112
NU_NOTA_CH                1112
NU_NOTA_LC                1170
TP_LINGUA                    0
TP_STATUS_REDACAO         1170
NU_NOTA_COMP1             1170
NU_NOTA_COMP2             1170
NU_NOTA_COMP3             1170
NU_NOTA_COMP4             1170
NU_NOTA_

In [7]:
teste['NU_NOTA_LC'].isna().sum()

1170

In [8]:
# Fazendo as trocas dos dados faltantes
for categorica in categoric_feature:
    
    treino[categorica].fillna(method='ffill', inplace=True)
    teste[categorica].fillna(method='ffill', inplace=True)
    
for categorica in categoric_feature:
    
    treino[categorica].fillna(method='bfill', inplace=True)
    teste[categorica].fillna(method='bfill', inplace=True)

for numerica in numeric_features:
        
    #ratings_petz_apple[f'{num_stars}-star_diff'].mask(cond=ratings_petz_apple[f'{num_stars}-star_diff'] < 0, other=0)
    media_treino = treino[numerica].mean()
    treino[numerica].fillna(0, inplace=True)
    
    media_teste = teste[numerica].mean()
    teste[numerica].fillna(0, inplace=True)

C:\Users\vinicius.oliveira\Anaconda3\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [9]:
# Conferindo se deu tudo certo com os dados de treino
treino.isna().sum().sum()

0

In [10]:
# Conferindo se deu tudo certo com os dados de teste
teste.isna().sum().sum()

0

In [11]:
# Instanciando um objeto da classe para o StandardScalar
std_scaler = StandardScaler()

In [12]:
# Aplicando o StandardScalar nas variáveis numéricas
treino[numeric_features] = std_scaler.fit_transform(treino[numeric_features])
teste[numeric_features] = std_scaler.fit_transform(teste[numeric_features])

C:\Users\vinicius.oliveira\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\vinicius.oliveira\Anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [13]:
# Conferindo os dados de treino
treino

,CO_UF_RESIDENCIA,NU_IDADE,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,TP_DEPENDENCIA_ADM_ESC,IN_BAIXA_VISAO,...,SG_UF_RESIDENCIA,TP_SEXO,Q001,Q002,Q006,Q024,Q025,Q026,Q027,Q047
0,1.243484,0.341624,-1.177119,-0.188775,-0.888684,0.576221,-0.633058,-0.582810,-0.630839,-0.035209,...,RS,M,D,D,C,A,A,C,H,A
1,-0.784493,-0.652215,0.808488,-0.188775,0.246774,-0.667092,1.157607,1.002369,1.159451,-0.035209,...,CE,F,A,A,B,A,A,A,H,A
2,-0.784493,-0.084307,0.808488,-0.188775,1.382232,-0.667092,-0.633058,-0.582810,-0.630839,-0.035209,...,CE,F,D,D,C,A,A,A,H,A
3,0.229495,0.483601,-2.169923,-0.188775,-0.888684,2.130362,-0.633058,-0.582810,-0.630839,-0.035209,...,RJ,F,H,E,E,C,B,C,F,D
4,-1.798482,0.909531,-0.184316,-0.188775,-0.888684,0.576221,-0.633058,-0.582810,-0.630839,-0.035209,...,AM,M,E,D,C,A,A,B,F,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13725,0.432293,-0.226284,-1.177119,-0.188775,-0.888684,0.265393,-0.633058,-0.582810,-0.630839,-0.035209,...,SP,M,E,E,G,B,B,B,C,A
13726,-0.480297,1.619416,0.808488,3.673938,-0.888684,2.441190,-0.633058,-0.582810,-0.630839,-0.035209,...,PE,F,A,A,C,A,B,B,I,A
13727,1.243484,-0.936169,-1.177119,-0.188775,1.382232,-0.667092,-0.633058,-0.582810,-0.630839,-0.035209,...,RS,F,C,G,B,A,B,A,I,A
13728,0.229495,2.045347,0.808488,-0.188775,2.517691,-0.667092,-0.633058,-0.582810,-0.630839,-0.035209,...,RJ,M,B,D,G,B,B,C,E,A


In [14]:
# Aplicando o get_dummies() nas variáveis categóricas
encoded_columns = pd.get_dummies(treino[categoric_feature])
treino = treino.join(encoded_columns).drop(categoric_feature, axis=1)

encoded_columns = pd.get_dummies(teste[categoric_feature])
teste = teste.join(encoded_columns).drop(categoric_feature, axis=1)


#ohe.fit(treino[categoric_feature])
#ohe.fit(teste[categoric_feature])

In [15]:
# Instanciando um objeto da classe LogisticRegression()
model = LogisticRegression()
treino = treino.join(treineiro)

In [16]:
# Separando X e y de treino
X_train = treino.drop('IN_TREINEIRO', axis=1)
y_train = treino['IN_TREINEIRO']

In [17]:
# Treinando o modelo
model.fit(X_train, y_train)

# Testando o modelo
y_test = model.predict(teste)

C:\Users\vinicius.oliveira\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [18]:
# Preparando a saída
y_test = pd.DataFrame(y_test)
y_test.rename(columns={0: 'IN_TREINEIRO'}, inplace=True)
y_test

,IN_TREINEIRO
0,0
1,0
2,1
3,0
4,0
...,...
4565,0
4566,0
4567,0
4568,0


In [19]:
# Juntando o número de inscrição aos dados preditos pelo modelo
my_answer = inscricao_teste.join(y_test)
my_answer

,NU_INSCRICAO,IN_TREINEIRO
0,ba0cc30ba34e7a46764c09dfc38ed83d15828897,0
1,177f281c68fa032aedbd842a745da68490926cd2,0
2,6cf0d8b97597d7625cdedc7bdb6c0f052286c334,1
3,5c356d810fa57671402502cd0933e5601a2ebf1e,0
4,df47c07bd881c2db3f38c6048bf77c132ad0ceb3,0
...,...,...
4565,361b7fcd8867119550fe2af5aa729ffad89a7cf5,0
4566,d8a0e4c9e29494cc9bba2422bd79333931475ee1,0
4567,3f1c3388244df8d6521e983a809292d9f3bca643,0
4568,1778e9c4cef591beb6b986d191d15ed05de816b0,0


In [20]:
my_answer.to_csv('answer.csv', index=False)